<a href="https://www.kaggle.com/code/priyankapalshetkar/kagglex-competition?scriptVersionId=117912181" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Steps to Consider:
* Add more algorithms to the mix
* Improve on the best algorithm (Random Forest, K-neighbors) by changing its hyperparameters (n_estimators, n_neighbors)
* Change the encoder
* Drop features which are not as important

# Load required libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import fbeta_score, accuracy_score,  make_scorer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from collections import defaultdict
from category_encoders.leave_one_out import LeaveOneOutEncoder
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import plotly.express as px
# from lazypredict.Supervised import LazyClassifier
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import catboost as cb

# SEED Everything : for reproducibility 

In [2]:
SEED = 42

def seed_everything(seed = 42):
    import random, os
    import numpy as np

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(SEED)

# Load dataset 

In [3]:
train_df = pd.read_csv('../input/kagglex-bipoc-2022-2023-ml-foundation/Train.csv')
test_df = pd.read_csv('../input/kagglex-bipoc-2022-2023-ml-foundation/Test.csv')
sample_sub = pd.read_csv('../input/kagglex-bipoc-2022-2023-ml-foundation/Sample_submission.csv')

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157509 entries, 0 to 157508
Data columns (total 42 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   ID        157509 non-null  object
 1   AAGE      157509 non-null  int64 
 2   ACLSWKR   157509 non-null  object
 3   ADTIND    157509 non-null  int64 
 4   ADTOCC    157509 non-null  int64 
 5   AHGA      157509 non-null  object
 6   AHRSPAY   157509 non-null  int64 
 7   AHSCOL    157509 non-null  object
 8   AMARITL   157509 non-null  object
 9   AMJIND    157509 non-null  object
 10  AMJOCC    157509 non-null  object
 11  ARACE     157509 non-null  object
 12  AREORGN   157509 non-null  object
 13  ASEX      157509 non-null  object
 14  AUNMEM    157509 non-null  object
 15  AUNTYPE   157509 non-null  object
 16  AWKSTAT   157509 non-null  object
 17  CAPGAIN   157509 non-null  int64 
 18  CAPLOSS   157509 non-null  int64 
 19  DIVVAL    157509 non-null  int64 
 20  FILESTAT  157509 non-null 

**My Initial Analysis:**

* What variables would affect my income?
AAGE (age), AHGA (education), AHRSPAY (Wage per hour), WKSWORK (Weeks worked in year), SEOTR (own business or self employed), CAPGAIN (capital gains), CAPLOSS (capital losses), AWKSTAT (full or part time employment stat), AUNTYPE (reason for unemployment), AMJIND (major industry code), AMJOCC (major occupation code), ACLSWK (class of worker), PRCITSHP (citizenship)

* Out of these which are the three most important ones?
AHRSPAY (Wage per hour), WKSWORK (Weeks worked in year),AWKSTAT (full or part time employment stat)

# Missing value analysis

In [5]:
missing_values_info = train_df.isnull().sum() / len(train_df)

In [6]:
missing_values_info_df = pd.DataFrame()
missing_values_info_df['features'] = missing_values_info.index
missing_values_info_df['missing_values'] = missing_values_info.values

In [7]:
px.bar(x='missing_values', y='features', data_frame=missing_values_info_df, title='Missing values in %', color='features')

* Features with null values?
GRINST (state of previous residence), MIGMTR1 (migration code change in msa), MIGMTR3 (migration code-change in reg), MIGMTR4 (migration code-move within reg), MIGSUN (migration prev res in sunbelt), PEFNTVTY (country of birth father), PEMNTVTY (country of birth mother), PENATVTY (country of birth self)

Since most of these fields are categorical, it won't be possible to guess what these values would be. So we can fill the NAs with category 'unknown'.

# Missing Values Imputation
* imputing with new category 'unknown'

In [8]:
selected_features = missing_values_info_df[missing_values_info_df['missing_values']>0]['features'].values

In [9]:
selected_features

array(['GRINST', 'MIGMTR1', 'MIGMTR3', 'MIGMTR4', 'MIGSUN', 'PEFNTVTY',
       'PEMNTVTY', 'PENATVTY'], dtype=object)

## Simple Imputer for filling in missing values?

In [10]:
for col in selected_features:
    train_df[col] = train_df[col].fillna('unknown')
    test_df[col] = test_df[col].fillna('unknown')

In [11]:
train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157509 entries, 0 to 157508
Data columns (total 42 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   ID        157509 non-null  object
 1   AAGE      157509 non-null  int64 
 2   ACLSWKR   157509 non-null  object
 3   ADTIND    157509 non-null  int64 
 4   ADTOCC    157509 non-null  int64 
 5   AHGA      157509 non-null  object
 6   AHRSPAY   157509 non-null  int64 
 7   AHSCOL    157509 non-null  object
 8   AMARITL   157509 non-null  object
 9   AMJIND    157509 non-null  object
 10  AMJOCC    157509 non-null  object
 11  ARACE     157509 non-null  object
 12  AREORGN   157509 non-null  object
 13  ASEX      157509 non-null  object
 14  AUNMEM    157509 non-null  object
 15  AUNTYPE   157509 non-null  object
 16  AWKSTAT   157509 non-null  object
 17  CAPGAIN   157509 non-null  int64 
 18  CAPLOSS   157509 non-null  int64 
 19  DIVVAL    157509 non-null  int64 
 20  FILESTAT  157509 non-null 

# Analysing the output variable

In [12]:
train_df['TARGET'].describe()

count    157509.000000
mean          0.082408
std           0.274986
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: TARGET, dtype: float64

In [13]:
income_greater_equal_to_50k = train_df['TARGET'].sum()
income_less_than_50k = train_df.shape[0] - income_greater_equal_to_50k
print(income_greater_equal_to_50k, income_less_than_50k)
print("% of people with salary greater than or equal to 50k", (100*income_greater_equal_to_50k/train_df.shape[0]).round(2), "%")

12980 144529
% of people with salary greater than or equal to 50k 8.24 %


**Only 8.24% of people have total income greater than or equal to 50k**

# LabelEncoder
* Converting categorical data to numerical form. LabelEncoder encode target labels with value between 0 and n_classes-1.

* using select_dtypes('object') for retreving string columns. select_dtypes returns a subset of the DataFrame’s columns based on the column dtypes

* [Reference](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) 

In [14]:
for col in train_df.select_dtypes('object'):
    if col != 'ID':
        le = LabelEncoder()
        train_df[col] = le.fit_transform(train_df[col])
        test_df[col] = le.transform(test_df[col])

In [15]:
train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157509 entries, 0 to 157508
Data columns (total 42 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   ID        157509 non-null  object
 1   AAGE      157509 non-null  int64 
 2   ACLSWKR   157509 non-null  int64 
 3   ADTIND    157509 non-null  int64 
 4   ADTOCC    157509 non-null  int64 
 5   AHGA      157509 non-null  int64 
 6   AHRSPAY   157509 non-null  int64 
 7   AHSCOL    157509 non-null  int64 
 8   AMARITL   157509 non-null  int64 
 9   AMJIND    157509 non-null  int64 
 10  AMJOCC    157509 non-null  int64 
 11  ARACE     157509 non-null  int64 
 12  AREORGN   157509 non-null  int64 
 13  ASEX      157509 non-null  int64 
 14  AUNMEM    157509 non-null  int64 
 15  AUNTYPE   157509 non-null  int64 
 16  AWKSTAT   157509 non-null  int64 
 17  CAPGAIN   157509 non-null  int64 
 18  CAPLOSS   157509 non-null  int64 
 19  DIVVAL    157509 non-null  int64 
 20  FILESTAT  157509 non-null 

# Prepare Train and Validation dataset

* Keeping 80% data for training and 20% for validation. 

* Using stratified approach to split the data. Stratified helps to keep distribution of target variable same for training and validation dataset. 

In [16]:
X = train_df.drop(columns=['ID','TARGET'])
print(X.shape)
y = train_df.TARGET
print(y.shape)

(157509, 40)
(157509,)


In [17]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y,
                                                        test_size=0.2, 
                                                        stratify=y, 
                                                        random_state=SEED)

In [18]:
X_train.shape, y_train.shape

((126007, 40), (126007,))

### target distribution in training 

In [19]:
y_train.value_counts()/len(y_train)

0    0.917592
1    0.082408
Name: TARGET, dtype: float64

In [20]:
X_valid.shape, y_valid.shape

((31502, 40), (31502,))

### target distribution in validation 

In [21]:
y_valid.value_counts()/len(y_valid)

0    0.917593
1    0.082407
Name: TARGET, dtype: float64

# Helper Function

In [22]:
def train_model(classifier, input_x, input_y):
    clf = classifier.fit(input_x, input_y)
    return clf

def evaluate_model(classifier, validation_x, validation_y, eval_metrics=fbeta_score):
    ypred = classifier.predict(validation_x)
    return ypred, eval_metrics(validation_y, ypred, beta=0.5)


# Comparing different ML algorithms

In [23]:
classifiers = {
    'logistic_regression' : LogisticRegression(solver='liblinear', random_state=SEED),
    'decision_tree' : DecisionTreeClassifier(random_state=SEED),
    'random_forest': RandomForestClassifier(random_state=SEED),
    # 'linear_svm': svm.SVC(kernel='linear',random_state=SEED),
    'naive_bayes': GaussianNB(),
    'k_neighbors': KNeighborsClassifier(),
    'LGBM': lgb.LGBMClassifier(objective="binary", random_state=SEED, n_estimators=250),
    'Gradient_Boosting_Classifier': GradientBoostingClassifier(random_state=SEED),
    'XGBoost' : XGBClassifier(random_state=SEED),
    'CatBoost' : cb.CatBoostClassifier(random_state=SEED)
    }

In [24]:
trained_models = {}
for classifier_name, classifier in classifiers.items():
    print("Started for: ", classifier)
    model = train_model(classifier, X_train, y_train)
    ypred, validation_score = evaluate_model(model, X_valid, y_valid)
    print("Done for: ", classifier)
    trained_models[classifier_name] = {'model': model, 'f1_score': validation_score}

Started for:  LogisticRegression(random_state=42, solver='liblinear')
Done for:  LogisticRegression(random_state=42, solver='liblinear')
Started for:  DecisionTreeClassifier(random_state=42)
Done for:  DecisionTreeClassifier(random_state=42)
Started for:  RandomForestClassifier(random_state=42)
Done for:  RandomForestClassifier(random_state=42)
Started for:  GaussianNB()
Done for:  GaussianNB()
Started for:  KNeighborsClassifier()
Done for:  KNeighborsClassifier()
Started for:  LGBMClassifier(n_estimators=250, objective='binary', random_state=42)
Done for:  LGBMClassifier(n_estimators=250, objective='binary', random_state=42)
Started for:  GradientBoostingClassifier(random_state=42)
Done for:  GradientBoostingClassifier(random_state=42)
Started for:  XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=

## Validation Results

In [25]:
validation_results = defaultdict(list)
for k,v in trained_models.items():
    validation_results['classifier_name'].append(k)
    validation_results['f1_score'].append(v['f1_score'])
validation_results = pd.DataFrame(validation_results)

In [26]:
validation_results

,classifier_name,f1_score
0,logistic_regression,0.537320
1,decision_tree,0.464469
2,random_forest,0.647754
3,naive_bayes,0.264062
4,k_neighbors,0.532417
5,LGBM,0.684717
6,Gradient_Boosting_Classifier,0.642107
7,XGBoost,0.680881
8,CatBoost,0.696980


In [27]:
px.bar(x='f1_score', y='classifier_name', data_frame=validation_results, color="classifier_name", title='Algorithm Performance Comparison')

# Inference :

* Piciking the best model for inference which is RandomForest

In [28]:
XTest = test_df.drop(columns=['ID'])

In [29]:
ytest_pred = trained_models["CatBoost"]["model"].predict(XTest)

In [30]:
output = sample_sub.copy()
output['TARGET'] = ytest_pred
output.head()

,ID,TARGET
0,ai1kagv30p8v,0
1,9s9e3x6a8f7u,0
2,qlvd7mszxd2z,0
3,uwhbqcnx5a5z,0
4,27c5sqbrzdwf,1


In [31]:
output['TARGET'].value_counts()/len(output)

0    0.948522
1    0.051478
Name: TARGET, dtype: float64

In [32]:
output.to_csv('./solution.csv', index=False)